In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.expanduser('~'), 'work', 'physion', 'src'))
import physion
import physion.utils.plot_tools as pt
import pandas as pd
import numpy as np

In [ ]:
datafolder = os.path.join(os.path.join(os.path.expanduser('~'), 'DATA', 'Cibele', 'PV_BB_V1', 'NWBs'))
protocol = 'ff-gratings-2orientations-8contrasts-15repeats'
DATASET = physion.analysis.read_NWB.scan_folder_for_NWBfiles(datafolder,
                                                  for_protocol=protocol, sorted_by='age')
df = pd.DataFrame({'file':[os.path.basename(f) for f in DATASET['files']],
              'subject':DATASET['subjects'],
              'protocol':[p[0] for p in DATASET['protocols']],
              'age':DATASET['ages']})

In [ ]:
# build summary pdf
class Null:
    def __init__(self, filename):
        self.datafile = filename

from physion.analysis.summary_pdf import generate_pdf
for i, f in enumerate(DATASET['files']):
    args = Null(f)
    generate_pdf(args, os.path.join(datafolder.replace('NWBs', 'pdfs'), 'A-%i.pdf' % (i+1)))

In [ ]:
from physion.analysis.protocols.contrast_sensitivity import compute_sensitivity_per_cells


stat_test = dict(interval_pre=[-1.5,-0.5],
                 interval_post=[0.5,1.5],
                 test='ttest',
                 positive=True)
response_significance_threshold=0.05

for angle in [0, 90]:
    
    for key in ['fraction_responsive', 'responsive',
                'tuning-curves']:
        DATASET[key+'-a=%.1f'%angle] = []
    
    for f in DATASET['files']:
        
        data = physion.analysis.read_NWB.Data(f, verbose=False)
        Episodes = physion.analysis.process_NWB.EpisodeData(data, quantities=['dFoF'], protocol_id=0, verbose=False)
    
        resp = compute_sensitivity_per_cells(data, Episodes,
                                                 angle= angle,
                                                 stat_test_props=stat_test,
                                                 response_significance_threshold=response_significance_threshold)
    
        DATASET['fraction_responsive'+'-a=%.1f'%angle].append(np.sum(resp['significant_ROIs'])/len(resp['significant_ROIs']))
        DATASET['responsive'+'-a=%.1f'%angle].append(resp['significant_ROIs'])
        DATASET['tuning-curves'+'-a=%.1f'%angle].append(np.clip(resp['Responses'], 0, np.inf))
    
DATASET['contrast'] = resp['contrast']

In [ ]:
ages = np.unique(DATASET['ages'])

Intervals = [[16,19],
             [20,23],
             [24,27]]


for angle in [0, 90]:
    
    fig, AX = pt.figure(axes=(len(Intervals), 1), wspace=2., hspace=0.6, top=2.)
    fig.suptitle('angle = %.i ' % angle)


    for i, age in enumerate(Intervals):
    
        cond = (DATASET['ages']>=age[0]) & (DATASET['ages']<=age[1])
    
        # responsiveness
        responsive = np.concatenate([r for r, c in zip(DATASET['responsive'+'-a=%.1f'%angle], cond) if c])
        r = np.sum(responsive)/len(responsive)
    
        inset = pt.inset(AX[i], [0.8,0.8,0.6,0.6])
        pt.pie([r, 1-r], COLORS=['tab:green', 'tab:grey'], ax=inset)
        inset.set_title('%.1f%%' %(100*r), color='tab:green')
        pt.annotate(AX[i], '%i ROIs\n' % len(responsive), (0, 1), ha='right')
    
        # tuning raw
        tuning = np.concatenate([r for r, c in zip(DATASET['tuning-curves'+'-a=%.1f'%angle], cond) if c])
        pt.plot(DATASET['contrast'], np.mean(tuning, axis=0), sy=np.std(tuning, axis=0), ax=AX[i])
    
        
        pt.set_plot(AX[i], title='P%i-P%i' % (age[0], age[1]), ylabel='$\\Delta$F/F', xlabel='angle from pref. ($^{o}$)')

In [ ]:
ages = np.unique(DATASET['ages'])

from scipy.stats import sem
Intervals = [[16,19],
             [20,23],
             [24,27]]


fig, AX = pt.figure(axes=(len(Intervals), 1), wspace=2., hspace=0.6, top=2.)
for i, age in enumerate(Intervals):
    for angle, color in zip([0, 90], ['tab:grey', 'k']):
    
        cond = (DATASET['ages']>=age[0]) & (DATASET['ages']<=age[1])
        # tuning normalized
        tuning = np.concatenate([r for r, c in zip(DATASET['tuning-curves'+'-a=%.1f'%angle], cond) if c])
        pt.plot(DATASET['contrast'], np.mean(tuning, axis=0), sy=sem(tuning, axis=0), color=color, ax=AX[i], ms=2)
        
        pt.annotate(AX[i], ' a=%.i'%angle, (.8,1), va='top', ha='right' if color!='k' else 'left', color=color, fontsize=7)
        
    pt.set_plot(AX[i], ylabel='norm. $\\Delta$F/F', xlabel='angle from pref. ($^{o}$)',
                title='P%i-P%i\n' % (age[0], age[1]), xticks=np.arange(3)*0.5)
#pt.set_common_ylims(AX)